In [1]:
import requests
import ast # abstract syntax tree
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# get brewery names and locations
URL= 'https://www.brewbound.com/breweries'

soup = BeautifulSoup(requests.get(URL).content, "html.parser")

data = soup.find_all('script')[-4]
string_results = str(data).split("\r\n")[2]
string_results_list = "["+string_results[:len(string_results) - 7]+"]"
results = ast.literal_eval(string_results_list)
df = pd.DataFrame(results, columns = ['brewery_name', 'latitude','longitude','url'])

df.head()

,brewery_name,latitude,longitude,url
0,Faust Hotel Restaurant and Brew PUB,29.701406,-98.123061,/breweries/Faust_Hotel_Restaurant_and_Brew_PUB
1,Wolf Pack Brewing Company,44.662053,-111.099461,/breweries/Wolf_Pack_Brewing_Co
2,Defiant Brewing Company,41.058405,-74.022847,/breweries/Defiant_Brewing_Co
3,El Toro Brewing Company,37.146552,-121.621987,/breweries/El_Toro_Brewing_Co
4,Sebago Brewing Company,43.679212,-70.396424,/breweries/Sebago_Brewing_Co


In [17]:
url = url = df['url'].iloc[[8]].to_string(header=False, index=False)
full_url = "https://www.brewbound.com"+url
page = requests.get(full_url, allow_redirects=False)
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find_all("address")

addr = []
for address in results:
    addr.append(address.text.strip())
print(addr)
print(page.text)

['McMinnville, OR']
<!DOCTYPE html>

<html>
<head>
    
    <title>Heater Allen Brewery | Brewbound.com</title>
    
    
		<meta charset="utf-8">

				<meta http-equiv="X-UA-Compatible" content="IE=edge">

		

				<meta name="HandheldFriendly" content="True">
		<meta name="MobileOptimized" content="320">
		<meta name="viewport" content="width=device-width, initial-scale=1.0">

		<link rel="apple-touch-icon" sizes="57x57" href="/Assets/img/apple-icon-57x57.png">
		<link rel="apple-touch-icon" sizes="60x60" href="/Assets/img/apple-icon-60x60.png">
		<link rel="apple-touch-icon" sizes="72x72" href="/Assets/img/apple-icon-72x72.png">
		<link rel="apple-touch-icon" sizes="76x76" href="/Assets/img/apple-icon-76x76.png">
		<link rel="apple-touch-icon" sizes="114x114" href="/Assets/img/apple-icon-114x114.png">
		<link rel="apple-touch-icon" sizes="120x120" href="/Assets/img/apple-icon-120x120.png">
		<link rel="apple-touch-icon" sizes="144x144" href="/Assets/img/apple-icon-144x144.png">
		<li

In [4]:
# use brewery url to get beers
beers = []
for i in range(len(df['url'])):
# for i in range(10):
    url = df['url'].iloc[[i]].to_string(header=False, index=False)
    print(i)
    full_url = "https://www.brewbound.com"+url
    try:
        page = requests.get(full_url, allow_redirects=False)
        soup = BeautifulSoup(page.content, "html.parser")
        addresses = soup.find_all("address")
        results = soup.find(id="beers")
    except:
        results = None
    
    addr = []
    for address in addresses:
        addr.append(address.text.strip())

    headers = []
    try:
        for j in results.find_all("th"):
            title = j.text
            headers.append(title)
        result_headers = headers[0:2]
        result_headers.append('url')
        result_headers.append('location')
    except AttributeError:
        print('No beers')
    data = []
    
    try:
        rows = results.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            cols = [ele for ele in cols if ele] # Get rid of empty values
            data.append([ele for ele in cols if ele != "No records to display."]) # Get rid of extra values
    
        table = list(filter(None, data))
        table = [x.append(url) or x for x in table]
        table = [x.append(addr) or x for x in table]
        beers= beers + table
    except AttributeError:
        print('No beers')

df_beers = pd.DataFrame(beers, columns = result_headers)

df_beers.head()

0
1
2
3
4
5
6
7
8
9


,Beer Name,Style,url,location
0,Very Hoppy Pale Ale,Pale Ale,/breweries/Defiant_Brewing_Co,"[Pearl River, NY]"
1,The Horseman's Ale,Ale,/breweries/Defiant_Brewing_Co,"[Pearl River, NY]"
2,Single Finger IPA,IPA,/breweries/Defiant_Brewing_Co,"[Pearl River, NY]"
3,Resolution #2,Belgian,/breweries/Defiant_Brewing_Co,"[Pearl River, NY]"
4,Prohibition Lager,Lager,/breweries/Defiant_Brewing_Co,"[Pearl River, NY]"


In [ ]:
# use beer url to get beer info